In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import json

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from google.colab import drive

##Configurazione

In [2]:
BASE_PATH ='./drive/MyDrive/Visione e Percezione/'

##Importazione Dataset

In [4]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
dataset = pd.DataFrame()
dataset_path = BASE_PATH + 'dataset/'
for file_name in os.listdir(dataset_path):
    file_path = dataset_path + file_name
    dataset_by_sign = pd.read_csv(file_path)
    dataset = dataset.append(dataset_by_sign)

In [29]:
dataset = dataset.sample(frac=1)
X = dataset.iloc[:, 0:32]
y = dataset.iloc[:, 32]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

##Training

###Tuning

In [ ]:
#clf = LogisticRegression(random_state=0).fit(X_train, y_train)
#print(f'Training Score: {clf.score(X_train, y_train)}')
#print(f'Test Score {clf.score(X_test, y_test)}%')

clf = LogisticRegression()
#solvers = ['newton-cg', 'lbfgs', 'liblinear']
solvers = ['newton-cg']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]
max_iter = [100, 200, 500, 1000]

grid = dict(solver=solvers, penalty=penalty, C=c_values, max_iter=max_iter)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=clf, param_grid=grid, cv=cv, scoring='accuracy', error_score=0, return_train_score=True)
grid_result = grid_search.fit(X_train, y_train)

In [ ]:
print(f"Miglior risultato: {grid_result.best_score_} con {grid_result.best_params_}")

Miglior risultato: 0.9368055555555554 con {'C': 100, 'max_iter': 100, 'penalty': 'l2', 'solver': 'newton-cg'}


BEST PARAMS: {'C': 100, 'max_iter': 100, 'penalty': 'l2', 'solver': 'newton-cg'}

###Modello

In [30]:
grid_search = LogisticRegression(C=100, max_iter=100, penalty='l2', solver='newton-cg')
grid_search.fit(X_train, y_train)

LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)

##Training & CV

In [ ]:
grid_result.cv_results_.pop('params', None)
pd.DataFrame(grid_result.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_max_iter,param_penalty,param_solver,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,split10_test_score,split11_test_score,split12_test_score,split13_test_score,split14_test_score,split15_test_score,split16_test_score,split17_test_score,split18_test_score,split19_test_score,split20_test_score,split21_test_score,split22_test_score,split23_test_score,split24_test_score,split25_test_score,split26_test_score,split27_test_score,split28_test_score,split29_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,split5_train_score,split6_train_score,split7_train_score,split8_train_score,split9_train_score,split10_train_score,split11_train_score,split12_train_score,split13_train_score,split14_train_score,split15_train_score,split16_train_score,split17_train_score,split18_train_score,split19_train_score,split20_train_score,split21_train_score,split22_train_score,split23_train_score,split24_train_score,split25_train_score,split26_train_score,split27_train_score,split28_train_score,split29_train_score,mean_train_score,std_train_score
0,4.942288,0.267024,0.002206,0.000093,100,100,l2,newton-cg,0.940625,0.926042,0.946875,0.930208,0.934375,0.939583,0.933333,0.947917,0.932292,0.933333,0.939583,0.939583,0.928125,0.940625,0.946875,0.946875,0.914583,0.952083,0.932292,0.927083,0.945833,0.933333,0.946875,0.939583,0.927083,0.934375,0.938542,0.935417,0.928125,0.942708,0.936806,0.008217,1,0.940625,0.941898,0.939815,0.940625,0.940509,0.940741,0.941782,0.939699,0.940972,0.941667,0.942014,0.939815,0.941551,0.940625,0.939815,0.939236,0.942708,0.939005,0.941319,0.942477,0.939236,0.940972,0.940741,0.939931,0.940625,0.942130,0.939931,0.940972,0.941551,0.941435,0.940814,0.000974
1,5.000543,0.360942,0.002568,0.001340,100,200,l2,newton-cg,0.940625,0.926042,0.946875,0.930208,0.934375,0.939583,0.933333,0.947917,0.932292,0.933333,0.939583,0.939583,0.928125,0.940625,0.946875,0.946875,0.914583,0.952083,0.932292,0.927083,0.945833,0.933333,0.946875,0.939583,0.927083,0.934375,0.938542,0.935417,0.928125,0.942708,0.936806,0.008217,1,0.940625,0.941898,0.939815,0.940625,0.940509,0.940741,0.941782,0.939699,0.940972,0.941667,0.942014,0.939815,0.941551,0.940625,0.939815,0.939236,0.942708,0.939005,0.941319,0.942477,0.939236,0.940972,0.940741,0.939931,0.940625,0.942130,0.939931,0.940972,0.941551,0.941435,0.940814,0.000974
2,4.893547,0.253855,0.002414,0.001195,100,500,l2,newton-cg,0.940625,0.926042,0.946875,0.930208,0.934375,0.939583,0.933333,0.947917,0.932292,0.933333,0.939583,0.939583,0.928125,0.940625,0.946875,0.946875,0.914583,0.952083,0.932292,0.927083,0.945833,0.933333,0.946875,0.939583,0.927083,0.934375,0.938542,0.935417,0.928125,0.942708,0.936806,0.008217,1,0.940625,0.941898,0.939815,0.940625,0.940509,0.940741,0.941782,0.939699,0.940972,0.941667,0.942014,0.939815,0.941551,0.940625,0.939815,0.939236,0.942708,0.939005,0.941319,0.942477,0.939236,0.940972,0.940741,0.939931,0.940625,0.942130,0.939931,0.940972,0.941551,0.941435,0.940814,0.000974
3,4.926450,0.273498,0.002208,0.000100,100,1000,l2,newton-cg,0.940625,0.926042,0.946875,0.930208,0.934375,0.939583,0.933333,0.947917,0.932292,0.933333,0.939583,0.939583,0.928125,0.940625,0.946875,0.946875,0.914583,0.952083,0.932292,0.927083,0.945833,0.933333,0.946875,0.939583,0.927083,0.934375,0.938542,0.935417,0.928125,0.942708,0.936806,0.008217,1,0.940625,0.941898,0.939815,0.940625,0.940509,0.940741,0.941782,0.939699,0.940972,0.941667,0.942014,0.939815,0.941551,0.940625,0.939815,0.939236,0.942708,0.939005,0.941319,0.942477,0.939236,0.940972,0.940741,0.939931,0.940625,0.942130,0.939931,0.940972,0.941551,0.941435,0.940814,0.000974
4,3.199071,0.135377,0.002204,0.000080,10,100,l2,newton-cg,0.894792,0.890625,0.906250,0.890625,0.892708,0.905208,

##Testing

In [24]:
map_labels = json.load(open(BASE_PATH + '/json/alphabet_mapping.json', 'rb'))

In [31]:
y_pred = grid_search.predict(X_test)
labels = np.unique(y_test)
conf_mat = confusion_matrix(y_test, y_pred, labels=labels)
labels = list(map(lambda x: map_labels[str(x)], labels))
pd.DataFrame(conf_mat, index=labels, columns=labels)
#N.B.: colonne previsioni, righe valori reali

,a,b,c,d,e,f,g,h,i,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,
a,87,0,1,1,0,0,0,0,0,0,0,0,0,3,0,1,0,1,0,0,0,0,0,1,0
b,0,88,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0
c,0,0,98,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
d,0,0,0,99,0,0,0,0,1,1,0,0,0,3,0,0,0,0,0,0,0,0,1,0,0
e,0,0,0,0,107,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
f,0,1,0,1,0,103,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
g,0,0,0,0,0,0,93,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
h,1,0,0,0,0,0,1,67,0,0,0,0,0,1,11,3,0,0,0,0,0,0,0,1,0
i,0,0,0,0,0,1,0,0,103,0,0,0,0,5,0,0,0,0,0,0,0,0,0,1,0
k,0,2,0,1,0,0,0,0,1,73,0,0,0,0,1,0,1,0,0,3,0,0,3,0,0


In [32]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.92      0.91        95
           1       0.92      0.97      0.94        91
           2       0.98      0.99      0.98        99
           3       0.93      0.94      0.93       105
           4       0.99      1.00      1.00       107
           5       0.99      0.97      0.98       106
           6       0.90      0.95      0.93        98
           7       0.73      0.79      0.76        85
           8       0.96      0.94      0.95       110
           9       0.94      0.86      0.90        85
          10       1.00      1.00      1.00        98
          11       0.91      1.00      0.95       107
          12       0.94      0.87      0.90        98
          13       0.85      0.89      0.87        91
          14       0.84      0.78      0.81       105
          15       0.84      0.61      0.70        94
          16       0.93      0.96      0.95        99
          17       0.95    

##Salvataggio

In [34]:
pickle.dump(grid_search, open(BASE_PATH + '/model/model.sav', 'wb'))